<a href="https://colab.research.google.com/github/RAGHURAM-SADINENI/Exploring-Ensembles-for-Object-Recognition/blob/main/GroupProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, activations
import numpy as np
import os


# Based on VGG-16 CNN.
# Perhaps we can have three models for ensemble: 1 shallow and large, one medium, one deep and small.
# Shallow and large will have fewer layers but more channels and larger kernel sizes.
# Deep and small will have more layers but less channels and 3x3 kernels.
class Model(keras.Model):
    def __init__(self, pool='max'):
        super(Model, self).__init__()

        if pool == 'max':
            self.pool1 = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same")
            self.pool2 = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same")
            self.pool3 = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same")
            self.pool4 = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same")
            self.pool5 = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same")
        elif pool == 'avg':
            self.pool1 = layers.AveragePooling2D(pool_size=(2,2), strides=(2,2), padding="same")
            self.pool2 = layers.AveragePooling2D(pool_size=(2,2), strides=(2,2), padding="same")
            self.pool3 = layers.AveragePooling2D(pool_size=(2,2), strides=(2,2), padding="same")
            self.pool4 = layers.AveragePooling2D(pool_size=(2,2), strides=(2,2), padding="same")
            self.pool5 = layers.AveragePooling2D(pool_size=(2,2), strides=(2,2), padding="same")

        # Convolutional and pooling layers.
        self.conv_layers = [
            layers.Conv2D(32, kernel_size=3, padding="same"),
            self.pool1,
            layers.Conv2D(64, kernel_size=3, padding="same"),
            self.pool2,
            layers.Conv2D(128, kernel_size=3, padding="same"),
            layers.Conv2D(128, kernel_size=3, padding="same"),
            self.pool3,
            layers.Conv2D(256, kernel_size=3, padding="same"),
            layers.Conv2D(256, kernel_size=3, padding="same"),
            self.pool4,
            layers.Conv2D(256, kernel_size=3, padding="same"),
            layers.Conv2D(256, kernel_size=3, padding="same"),
            self.pool5
        ]

        self.dense_layers = [
            layers.Dense(1024),
            layers.Dense(128)
        ]

        # Original VGG.
        # self.conv_layers = [
        #     layers.Conv2D(64, kernel_size=3, padding="same"),
        #     layers.Conv2D(64, kernel_size=3, padding="same"),
        #     self.pool1,
        #     layers.Conv2D(128, kernel_size=3, padding="same"),
        #     layers.Conv2D(128, kernel_size=3, padding="same"),
        #     self.pool2,
        #     layers.Conv2D(256, kernel_size=3, padding="same"),
        #     layers.Conv2D(256, kernel_size=3, padding="same"),
        #     layers.Conv2D(256, kernel_size=3, padding="same"),
        #     self.pool3,
        #     layers.Conv2D(512, kernel_size=3, padding="same"),
        #     layers.Conv2D(512, kernel_size=3, padding="same"),
        #     layers.Conv2D(512, kernel_size=3, padding="same"),
        #     self.pool4,
        #     layers.Conv2D(512, kernel_size=3, padding="same"),
        #     layers.Conv2D(512, kernel_size=3, padding="same"),
        #     layers.Conv2D(512, kernel_size=3, padding="same"),
        #     self.pool5
        # ]

        # self.dense_layers = [
        #     layers.Dense(4096),
        #     layers.Dense(4096)
        # ]

        self.decode_layer = layers.Dense(10, activation='softmax')


    def call(self, x):
        for l in self.conv_layers:
          x = l(x)

        for l in self.dense_layers:
          x = l(x)

        return self.decode_layer(x)


#   Load mnist, divide by 255 to convert values to be between zero and one.
def load_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='mnist.npz')
    (x_train, y_train), (x_test, y_test) = (tf.expand_dims(x_train.astype("float32") / 255, -1), y_train.astype("float32")), (tf.expand_dims(x_test.astype("float32") / 255, -1), y_test.astype("float32"))

    return (x_train, y_train), (x_test, y_test)


def train_model(x_train, y_train, x_test, y_test):

    model = Model()

    model.compile(
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            metrics=["accuracy"]
    )

    epoch = 1
    batch_size = 64

    print(os.getcwd())

    model.fit(x_train, y_train, batch_size=batch_size, epochs=1)
    model.save_weights("/content/cnn_model1_1ep")
    model.evaluate(x_test, y_test, batch_size=batch_size)


(x_train, y_train), (x_test, y_test) = load_data()
train_model(x_train, y_train, x_test, y_test)

/content
938/938 [==============================] - 539s 574ms/step - loss: 0.2208 - accuracy: 0.1136


ValueError: ignored